In [51]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

In [52]:
#  Define the data directory and other hyperparameters
train_dir = 'final_dataset/train'
val_dir = 'final_dataset/validation'
test_dir = 'final_dataset/test'
batch_size = 64
input_shape = (227, 227, 3)
num_classes = 101
num_epochs = 30


In [53]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255
#     rotation_range=15,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Do not shuffle the test data
)


Found 75750 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.
Found 15150 images belonging to 101 classes.


In [54]:
# Load the pre-trained AlexNet model with weights from ImageNet
base_model = tf.keras.applications.AlexNet(
    include_top=False, weights='imagenet', input_shape=input_shape)

# Freeze the pre-trained layers (optional)
for layer in base_model.layers:
    layer.trainable = False

AttributeError: module 'keras.api._v2.keras.applications' has no attribute 'AlexNet'

In [ ]:
# Add custom classification layers on top of the pre-trained model
x = layers.Flatten()(base_model.output)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


In [47]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 55, 55, 96)        34944     
                                                                 
 max_pooling2d_20 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 flatten_9 (Flatten)         (None, 69984)             0         
                                                                 
 dense_19 (Dense)            (None, 256)               17916160  
                                                                 
 dense_20 (Dense)            (None, 101)               25957     
                                                                 
Total params: 17,977,061
Trainable params: 17,977,061
Non-trainable params: 0
__________________________________________

In [49]:
# Define callbacks
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    'alexnet_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

early_stopping_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)


In [55]:
# Training the model
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

Epoch 1/30
1184/1184 [==============================] - ETA: 0s - loss: 3.8667 - accuracy: 0.1249
Epoch 1: val_loss did not improve from 4.25481
1184/1184 [==============================] - 156s 132ms/step - loss: 3.8667 - accuracy: 0.1249 - val_loss: 4.3480 - val_accuracy: 0.0683
Epoch 2/30
1184/1184 [==============================] - ETA: 0s - loss: 3.6243 - accuracy: 0.1667
Epoch 2: val_loss did not improve from 4.25481
1184/1184 [==============================] - 159s 135ms/step - loss: 3.6243 - accuracy: 0.1667 - val_loss: 4.4838 - val_accuracy: 0.0672
Epoch 3/30
1184/1184 [==============================] - ETA: 0s - loss: 3.1841 - accuracy: 0.2513
Epoch 3: val_loss did not improve from 4.25481
1184/1184 [==============================] - 158s 133ms/step - loss: 3.1841 - accuracy: 0.2513 - val_loss: 4.8306 - val_accuracy: 0.0648
Epoch 4/30
1184/1184 [==============================] - ETA: 0s - loss: 2.5317 - accuracy: 0.3893
Epoch 4: val_loss did not improve from 4.25481
1184/1184

In [ ]:
 Evaluation on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2%}")

In [ ]:
# Calculate metrics
test_generator.reset()
predictions = model.predict(test_generator)
y_pred = [tf.argmax(pred).numpy() for pred in predictions]
y_true = test_generator.classes

In [ ]:
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))
print("Accuracy: {:.2f}%".format(100 * accuracy_score(y_true, y_pred)))
print("F1 Score: {:.2f}".format(f1_score(y_true, y_pred, average='macro')))
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred)
